In [3]:
from spieks.neurons import SpikingNeuron, IF
from spieks.network import run_sim
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import DataLoader

N = 512
T = 1
DT = 1e-2

MAX_HZ = 10

BS = 128

In [4]:
transform = torchvision.transforms.Compose([
	torchvision.transforms.ToTensor(),
	torchvision.transforms.Normalize(
	(0.1307,), (0.3081,))
]),
train_loader = DataLoader(torchvision.datasets.MNIST('./tmp/', train=True, download=True, transform=transform), batch_size=BS, shuffle=True)
test_loader = DataLoader(torchvision.datasets.MNIST('./tmp/', train=False, download=True, transform=transform), batch_size=BS, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./tmp/MNIST/raw/train-images-idx3-ubyte.gz to ./tmp/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./tmp/MNIST/raw/train-labels-idx1-ubyte.gz to ./tmp/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./tmp/MNIST/raw/t10k-images-idx3-ubyte.gz to ./tmp/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./tmp/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./tmp/MNIST/raw



In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)